# Natural Language Processing (688 only) [35pts]
## Introduction

In this problem you will develop two techniques for analyzing free text documents: a bag of words approach based upon creating a TFIDF matrix, and an n-gram language model.

You'll be applying your models to the text from the Federalist Papers.  The Federalist papers were a series of essay written in 1787 and 1788 by Alexander Hamilton, James Madison, and John Jay (they were published anonymously at the time), that promoted the ratification of the U.S. Constitution.  If you're curious, you can read more about them here: https://en.wikipedia.org/wiki/The_Federalist_Papers . They are a particularly interesting data set, because although the authorship of most of the essays has been long known since around the deaths of Hamilton and Madison, there was still some question about the authorship of certain articles into the 20th century.  You'll use document vectors and language models to do this analysis for yourself.

## The dataset

You'll use a copy of the Federalist Papers downloaded from Project Guttenberg, available here: http://www.gutenberg.org/ebooks/18 .  Specifically, the "pg18.txt" file included with the homework is the raw text downloaded from Project Guttenberg.  To ensure that everyone starts with the exact same corpus, we are providing you the code to load and tokenize this document, as given below.

In [1]:
import re

def load_federalist_corpus(filename):
    """ Load the federalist papers as a tokenized list of strings, one for each eassay"""
    with open(filename, "rt") as f:
        data = f.read()
    papers = data.split("FEDERALIST")
    
    # all start with "To the people of the State of New York:" (sometimes . instead of :)
    # all end with PUBLIUS (or no end at all)
    locations = [(i,[-1] + [m.end()+1 for m in re.finditer(r"of the State of New York", p)],
                 [-1] + [m.start() for m in re.finditer(r"PUBLIUS", p)]) for i,p in enumerate(papers)]
    papers_content = [papers[i][max(loc[1]):max(loc[2])] for i,loc in enumerate(locations)]

    # discard entries that are not actually a paper
    papers_content = [p for p in papers_content if len(p) > 0]

    # replace all whitespace with a single space
    papers_content = [re.sub(r"\s+", " ", p).lower() for p in papers_content]

    # add spaces before all punctuation, so they are separate tokens
    punctuation = set(re.findall(r"[^\w\s]+", " ".join(papers_content))) - {"-","'"}
    for c in punctuation:
        papers_content = [p.replace(c, " "+c+" ") for p in papers_content]
    papers_content = [re.sub(r"\s+", " ", p).lower().strip() for p in papers_content]
    
    authors = [tuple(re.findall("MADISON|JAY|HAMILTON", a)) for a in papers]
    authors = [a for a in authors if len(a) > 0]
    
    numbers = [re.search(r"No\. \d+", p).group(0) for p in papers if re.search(r"No\. \d+", p)]
    
    return papers_content, authors, numbers
    
    

In [13]:
### AUTOLAB_IGNORE_START
# papers, authors, numbers = load_federalist_corpus("pg18.txt")
### AUTOLAB_IGNORE_STOP

You're welcome to dig through the code here if you're curious, but it's more important that you look at the objects that the function returns.  The `papers` object is a list of strings, each one containing the full content of one of the Federalist Papers.  All tokens (words) in the text are separated by a single space (this includes some puncutation tokens, which have been modified to include spaces both before and after the punctuation. The `authors` object is a list of lists, which each list contains the author (or potential authors) of a given paper.  Finally the `numbers` list just contains the number of each Federalist paper.  You won't need to use this last one, but you may be curious to compare the results of your textual analysis to the opinion of historians.

## Q1: Bag of words, and TFIDF [6+6pts]

In this portion of the question, you'll use a bag of words model to describe the corpus, and write routines to build a TFIDF matrix and a cosine similarity function.  Specifically, you should first implement the TFIDF function below.  This should return a _sparse_ TFIDF matrix (as for the Graph Library assignment, make sure to directly create a sparse matrix using `scipy.sparse.coo_matrix()` rather than create a dense matrix and then convert it to be sparse).

Important: make sure you do _not_ include the empty token `""` as one of your terms. 

In [3]:
import collections # optional, but we found the collections.Counter object useful
import scipy.sparse as sp
import numpy as np 

def tfidf(docs):
    """
    Create TFIDF matrix.  This function creates a TFIDF matrix from the
    docs input.

    Args:
        docs: list of strings, where each string represents a space-separated
              document
    
    Returns: tuple: (tfidf, all_words)
        tfidf: sparse matrix (in any scipy sparse format) of size (# docs) x
               (# total unique words), where i,j entry is TFIDF score for 
               document i and term j
        all_words: list of strings, where the ith element indicates the word
                   that corresponds to the ith column in the TFIDF matrix
    """
    vocab = {}
    df = {}
    regex = re.compile("\s+")
    count = 0
    for doc in docs:
        terms = re.split(regex, doc)
        for term in set(terms):
            if len(term) > 0:
                if term not in vocab:
                    vocab[term] = count  # (index, df)
                    df[term] = 1
                    count += 1
                else:
                    df[term] += 1
    num_docs = len(docs)
    scores = []
    for i in range(0, num_docs):
        scores.append({})

    for index in range(0, num_docs):
        terms = re.split(regex, docs[index])
        for term, tf in collections.Counter(terms).most_common():
            if len(term) > 0:
                term_index = vocab[term]
                score = float(tf) * np.log(float(num_docs) / float(df[term]))
                if score > 0.0:
                    scores[index][term_index] = score

    i_list = []
    j_list = []
    data = []

    for i in range(0, num_docs):
        for j, score in scores[i].iteritems():
            i_list.append(i)
            j_list.append(j)
            data.append(score)

    matrix = sp.csr_matrix((data, (i_list, j_list)), shape=(num_docs, len(vocab)))
    reverse_map = {v: k for k, v in vocab.iteritems()}
    return matrix, reverse_map.values()

In [4]:
### AUTOLAB_IGNORE_START

# X_tfidf, words = tfidf(papers)
# print type(X_tfidf)
# print type(words)
# print len(X_tfidf.nonzero()[0])
# print len(words)

### AUTOLAB_IGNORE_STOP

Our version results the following result (just showing the type, size, and # of non-zero elements):

    <86x8686 sparse matrix of type '<type 'numpy.float64'>'
        with 57607 stored elements in Compressed Sparse Row format>
     
For testing, you can also run the algorithm on the following "data set" from class:

In [5]:
### AUTOLAB_IGNORE_START
# data = [
#     "the goal of this lecture is to explain the basics of free text processing",
#     "the bag of words model is one such approach",
#     "text processing via bag of words"
# ]

# X_tfidf, words = tfidf(data)
# print X_tfidf.todense()
# print words
# print len(X_tfidf.nonzero()[0])

### AUTOLAB_IGNORE_STOP

For our implementation, this returns the following output:

    [[ 0.          0.          1.09861229  1.09861229  0.          1.09861229
       0.          0.40546511  0.40546511  1.09861229  0.          1.09861229
       0.          0.          0.40546511  1.09861229  0.81093022  0.
       1.09861229]
     [ 1.09861229  1.09861229  0.          0.          0.40546511  0.          0.
       0.40546511  0.          0.          1.09861229  0.          0.
       0.40546511  0.          0.          0.40546511  1.09861229  0.        ]
     [ 0.          0.          0.          0.          0.40546511  0.          0.
       0.          0.40546511  0.          0.          0.          1.09861229
       0.40546511  0.40546511  0.          0.          0.          0.        ]]
    ['model', 'such', 'basics', 'goal', 'bag', 'this', 'of', 'is', 'processing', 'free', 'one', 'to', 'via', 'words', 'text', 'lecture', 'the', 'approach', 'explain']

Next, implement the following simply function that takes the X_tfidf matrix (though it could also take simple term frequency matrices, etc), and compute a matrix of all pair-wise cosine similarities.

In [6]:
def cosine_similarity(X):
    """
    Return a matrix of cosine similarities.
    
    Args:
        X: sparse matrix of TFIDF scores or term frequencies
    
    Returns:
        M: dense numpy array of all pairwise cosine similarities.  That is, the 
           entry M[i,j], should correspond to the cosine similarity between the 
           ith and jth rows of X.
    """ 
    matrix = X.dot(X.transpose()).todense()
    mat_len = len(matrix)
    norms = [0] * mat_len
    for i in range(0, mat_len):
        norms[i] = 1.0 / np.sqrt(matrix.item((i, i)))
    norm_mat = np.matrix(norms)
    return np.multiply(norm_mat.transpose().dot(norm_mat), matrix)

In [7]:
### AUTOLAB_IGNORE_START
# print cosine_similarity(X_tfidf)
### AUTOLAB_IGNORE_STOP

If you apply this function to the example from class:

    M = cosine_similarity(X_tfidf)
    
we get the result presented in the slides:

    [[ 1.          0.06796739  0.07771876]
     [ 0.06796739  1.          0.10281225]
     [ 0.07771876  0.10281225  1.        ]]

Finally, use this model to analyze potential authorship of the unknown Federalist Papers.  Specifically, compute the average cosine similarity between all the _known_ Hamilton papers and all the _unknown_ papers (and similarly between known Madison and unknown, and Jay and unknown).  Populate the following variables with these averages.  As a quick check, our value for the `jay_mean_cosine_similarity` (and we know definitively that the unknown papers were _not_ written by Jay), is 0.064939.

In [14]:
### AUTOLAB_IGNORE_START
# hamilton = []
# madison = []
# jay = []
# unknown = []

# h_indexes = []
# m_indexes = []
# j_indexes = []
# u_indexes = []

# index = -1
# for p, a in zip(papers, authors):
#     index += 1
#     if len(a) > 1:
#         unknown.append(p)
#         u_indexes.append(index)
#         continue
#     if 'HAMILTON' in a:
#         hamilton.append(p)
#         h_indexes.append(index)
#     if "MADISON" in a:
#         madison.append(p)
#         m_indexes.append(index)
#     if "JAY" in a:
#         jay.append(p)
#         j_indexes.append(index)

# print "u_len: ", len(u_indexes), " j_len: ", len(j_indexes) 
        
# X_tfidf, words = tfidf(papers)
# similarity = cosine_similarity(X_tfidf)

# hamilton_mean_cosine_similarity = 0.0 
# madison_mean_cosine_similarity = 0.0
# jay_mean_cosine_similarity = 0.0

# for i in u_indexes:
#     for j in h_indexes:
#         hamilton_mean_cosine_similarity += similarity.item((i, j))
#     for j in m_indexes:
#         madison_mean_cosine_similarity += similarity.item((i, j))
#     for j in j_indexes:
#         jay_mean_cosine_similarity += similarity.item((i, j))
        
# hamilton_mean_cosine_similarity /= float(len(u_indexes) * len(h_indexes))
# madison_mean_cosine_similarity /= float(len(u_indexes) * len(m_indexes))
# jay_mean_cosine_similarity /= float(len(u_indexes) * len(j_indexes))

# print jay_mean_cosine_similarity
### AUTOLAB_IGNORE_STOP

## Q2: N-gram language models [0+10+13pts]

In this question, you will implement an n-gram model to be able to model the language used in the Federalist Papers in a more structured manner than the simple bag of words approach.  You will fill in the following class:

In [33]:
import random
import math

class LanguageModel:
    def __init__(self, docs, n):
        """
        Initialize an n-gram language model.

        Args:
            docs: list of strings, where each string represents a space-separated
                  document
            n: integer, degree of n-gram model
        """
        self.n = n
        self.dict = {}
        self.vocab = set()
        self.sum_index = "**sum*"
        regex = re.compile("\s+")
        count = 0
        for doc in docs:
            terms = re.split(regex, doc)
            self.vocab = self.vocab.union(set(terms))
            for i in range(0, len(terms) - n + 1):
                end = i+n-1
                t = tuple(terms[i:end])
                if t not in self.dict:
                    self.dict[t] = {}
                    self.dict[t][self.sum_index] = 0
                self.dict[t][self.sum_index] += 1
                end_term = terms[end]
                if end_term not in self.dict[t]:
                    self.dict[t][end_term] = 1
                else:
                    self.dict[t][end_term] += 1

    def perplexity(self, text, alpha=1e-3):
        """
        Evaluate perplexity of model on some text.

        Args:
            text: string containing space-separated words, on which to compute
            alpha: constant to use in Laplace smoothing

        Note: for the purposes of smoothing, the dictionary size (i.e, the D term)
        should be equal to the total number of unique words used to build the model
        _and_ in the input text to this function.

        Returns: perplexity
            perplexity: floating point value, perplexity of the text as evaluted
                        under the model.
        """        
        regex = re.compile("\s+")
        terms = re.split(regex, text)
        n = self.n
        logp_sum = 0.0    
        D = len(self.vocab.union(set(terms)))

        for i in range(0, len(terms) - n + 1):
            end = i + n - 1
            t = tuple(terms[i:end])
            end_term = terms[end]
            c = 0.0
            c_sum = 0.0
            if t in self.dict:
                c_sum = self.dict[t][self.sum_index]
                if end_term in self.dict[t]:
                    c = self.dict[t][end_term]
                
            log_p = np.log2(float(c + alpha)) - np.log2(float(c_sum + alpha * D))
            logp_sum += log_p
        return np.power(2, -logp_sum/(len(terms) - n + 1))

    def gen_beginning(self):
        t_sum = len(self.dict)
        rand = random.randint(0, t_sum)
        i = 0
        for t in self.dict.iterkeys():
            if i == rand:
                return list(t)
            i += 1

    def sample(self, k):
        """
        Generate a random sample of k words.

        Args:
            k: integer, indicating the number of words to sample

        Returns: text
            text: string of words generated from the model.
        """
        result = []
        current = self.gen_beginning()
        while len(result) < k:
            t = tuple(current)
            if t in self.dict:
                c_sum = self.dict[t][self.sum_index]
                rand = random.randint(0, c_sum)
                new_term = ""
                for term, count in self.dict[t].iteritems():
                    if term == self.sum_index:
                        continue
                    if rand > count:
                        rand -= count
                    else:
                        new_term = term
                        break
                result.append(current[0])
                current.remove(current[0])
                current.append(new_term)
            else:
                current = self.gen_beginning()
        return " ".join(result)

### Part A: Initializing the language model

First, implement the `__init__()` function in the `LanguageModel` class.  You can store the information any way you want, but we did this by building a two-level dictionary (in fact, we used the `collections.defaultdict` class, but this only make a few things a little bit shorter ... you are free to use it or not) `self.counts`, where the first key refers to the previous $n-1$ tokens, and the second key refers to the $n$th token, and the value simply stores the count of the number of times this combination was seen.  For ease of use in later function, we also created a `self.count_sums`, which contained the number of total times each $n-1$ combination was seen.

For example, letting `l_hamilton` be a LanguageModel object built just from all the known Hamilton papers and with `n = 3`, the following varibles are populated in the object:

    l_hamilton.counts["privilege of"] = {'being': 1, 'originating': 1, 'paying': 1, 'the': 1}
    l_hamilton.count_sums["privilege of"] = 4
    
We also build a `self.dictionary` variable, which is just a `set` object containing all the unique words in across the entire set of input document.

### Part B: Computing perplexity

Next, implement the `perplexity()` function, which takes a text sample and computes the perplexity of this sample under the model.  Use the formula for perplexity from the class nodes (which is actually not exact, since it only so, being careful to not multiply togther probabilites that get too small (hint: instead of taking the log of a large product, take the sum of the logs).

You'll want to be careful about dictionary sizes when it comes to the Laplace smoothing term: make sure your dictionary size $D$ is equal to the total number of unique words that occur in either the original data used to build the language model _or_ in the text we are evaluating the perplexity of (so the size of the union of the two).

As a simple example, if we build our `l_hamilton` model above (again, with `n=3`) and using default settings so that `alpha = 1e-3`, and run in on `papers[0]` (which was written by Hamilton), we get:

    l_hamilton.perplexity(papers[0]) = 12.5877

Using this model, evaluate the mean of the perplexity of the unknown Federalist papers for the language models from each of the three authors (again, using `n=3` and the default of `alpha=1e-3`).  Populate the following variables with the mean perplexities.

In [34]:
### AUTOLAB_IGNORE_START
# l_hamilton = LanguageModel(hamilton, 3)
# l_madison = LanguageModel(madison, 3)
# l_jay = LanguageModel(jay, 3)

# perp_hamilton = 0.0
# perp_madison = 0.0
# perp_jay = 0.0

# print l_hamilton.perplexity(papers[1])

# for u in u_indexes:
#     perp_hamilton += l_hamilton.perplexity(papers[u])
#     perp_madison += l_madison.perplexity(papers[u])
#     perp_jay += l_jay.perplexity(papers[u])

# perp_hamilton /= float(len(u_indexes))
# perp_madison /= float(len(u_indexes))
# perp_jay /= float(len(u_indexes))

# print perp_hamilton
### AUTOLAB_IGNORE_STOP

2671.54703136


In [16]:
### AUTOLAB_IGNORE_START
# print l_hamilton.dict[("privilege", "of")]
### AUTOLAB_IGNORE_STOP

{'being': 1, 'originating': 1, 'the': 1, '**sum*': 4, 'paying': 1}


As a check, you should get a perplexity of 1941.38516773 for the unknown papers under the Hamilton model. 

### Part C: Generating text

Finally, implement the `sample()` function to generate random samples of text.  Essentially, you want to pick some random starting $n-1$ tuples (you can do this any way you want), then sample according to the model.  Here you should _not_ use any Laplace smoothing, but just sample from the raw underlying counts.

One potential failure case, since we're just using raw counts, is if we generate an n-gram that _only_ occurs at the very end of a document (and so has no following n-gram observed in the data).  In this happens, just generate a new random set of $n-1$ tuples, and continue generating.

We'll be pretty lax in grading here: we're just going to be evaluating the perplexity of the generated text and make sure it's within some very loose bounds.  This is more for the fun of seeing some generated text than for actual concrete evaluation, since it's generating a random sample.  Here's what a sample of 200 words from our Hamilton model looks like (of course all random samples will be different). 

    'erroneous foundation . the authorities essential to the mercy of the body politic against these two legislatures coexisted for ages in two ways : either by actual possession of which , if it cease to regard the servile pliancy of the states which have a salutary and powerful means , by those who appoint them . they are rather a source of delinquency , it would enable the national situation , and consuls , judges of their own immediate aggrandizement ? would she not have been felt by those very maxims and councils which would be mutually questions of property ? and will he not only against the united netherlands with that which has been presumed . the censure attendant on bad measures among a multitude that might have been of a regular and effective system of civil polity had previously established in this state , but upon our neutrality . by thus circumscribing the plan of opposition , and the industrious habits of the trust committed to hands which could not be likely to do anything else . when will the time at which we might soar to a deed for conveying property of the people , were dreaded and detested'



In [37]:
### AUTOLAB_IGNORE_START
# print l_hamilton.sample(200)
# print len(l_hamilton.sample(200).split())
# l_hamilton.perplexity(l_hamilton.sample(200))
### AUTOLAB_IGNORE_STOP

claimed under the name of government , and especially from all sincere lovers of their navigation and the want of being taught by the convention , in respect to elections , contained in the matter cannot fail to agitate the passions and their constitution prevents the differences of opinion , which have been no direct contradiction of power ; and that the joint efforts of the state , unless where they reside , more foundation , the principle to which it is evident that a continuance of his treachery to his own capacity and desert ; and these effects would be immediately injured by the convention , it will readily be perceived that a provision of this nature , is impossible to believe that they would affect the spirit of the country at and near the seat of war , and of other irregular and violent propensities ? is it not ( we may safely count upon their pride , the fate of a few hours , or in part , they may have led to conclude without first taking a survey of the person , in every con

12.342513693469574

{'**sum*': 2, ',': 1, 'that': 1}
